In [1]:
from keras import backend as K
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Input, Lambda, concatenate
import numpy as np
from sklearn.utils import shuffle

from keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.losses import binary_crossentropy
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from scipy.spatial import distance
import keras
from keras.datasets import cifar100

Using TensorFlow backend.


### Task 2.1: triplet siamese network training

In [26]:
# get Cifar-100 data
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

n_classes = 100 # given
input_shape = x_test.shape[1:]

# get indices of sorted data so we can order out data by category
idx = np.argsort(y_train, axis=None)

x_train = x_train[idx].reshape(n_classes, -1, *input_shape)
y_train = y_train[idx].reshape(n_classes, -1, 1)

# select 80 classes for training data
x_train_trip = x_train[:80]
y_train_trip = y_train[:80]

# select remaining 20 classes for test data
x_test_trip = x_train[80:]
y_test_trip = y_train[80:]

# check shapes
print(x_train_trip.shape)
print(y_train_trip.shape)
print(x_test_trip.shape)
print(y_test_trip.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(80, 500, 32, 32, 3)
(80, 500, 1)
(20, 500, 32, 32, 3)
(20, 500, 1)


In [0]:
def triplet_loss(ground_truth, network_output):
    anchor, positive, negative = tf.split(network_output, num_or_size_splits=3, axis=1)
    
    for embedding in [anchor, positive, negative]:
        embedding = tf.math.l2_normalize(embedding)
    
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=1)
    
    margin = 0.2 # define your margin, 0.2 filled in now so it compilesss and so we can test stuff
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), margin)
    
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0), axis=0)
    
    return loss

In [0]:
def get_batch(batch_size, X):
    """Create batch of n triples, with positive and negatives"""
    n_classes, n_examples, w, h, d = X.shape
    # randomly sample several classes to use in the batch
    categories = np.random.choice(n_classes, size=(batch_size,), replace=False)
    # initialize 3 empty arrays for the input image batch, where triplets[0] is the anchor
    # triplets[1] is the positive match to the anchor, and pairs[2] is the negative match to the anchor
    triplets = [np.zeros((batch_size, h, w, d)) for i in range(3)]
    # initialize vector for the targets, and make one half of it '1's, so 2nd half of batch has same class
#     targets = np.zeros((batch_size,))
#     targets[batch_size//2:] = 1
    targets = np.zeros((batch_size,))
    for i in range(batch_size):
      category = categories[i]
      idx_1 = np.random.randint(0, n_examples)
      triplets[0][i, :, :, :] = X[category, idx_1].reshape(w, h, d)
      idx_2 = np.random.randint(0, n_examples)
      triplets[1][i, :, :, :] = X[category, idx_2].reshape(w, h, d)
      idx_3 = np.random.randint(0, n_examples)
      category_2 = (category + np.random.randint(1,n_classes)) % n_classes
      triplets[2][i, :, :, :] = X[category_2, idx_3].reshape(w, h, d)
#       print(category, category_2)
    return triplets, targets
  
def batch_generator(batch_size, X):
  while True:
    triplets, targets = get_batch(batch_size, X)
    yield (triplets, targets)
  
  
def train(model, X_train, batch_size=64, steps_per_epoch=100,epochs=1):
  model.fit_generator(batch_generator(batch_size, X_train),steps_per_epoch=steps_per_epoch, epochs=epochs)
  
  

In [0]:
import tensorflow as tf
def triplet_loss(ground_truth, network_output):

    anchor, positive, negative = tf.split(network_output, num_or_size_splits=3, axis=1)        

    for embedding in [anchor, positive, negative]:
        embedding = tf.math.l2_normalize(embedding)

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=1)

    margin = 0.15 # define your margin
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), margin)
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0), axis=0)

    return loss

In [31]:
input_shape = (32, 32, 3)
anchor_input = Input(input_shape)
positive_input = Input(input_shape)
negative_input = Input(input_shape)

convnet = Sequential()
convnet.add(Conv2D(128, (3, 3), activation='relu', input_shape=input_shape,  kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))

convnet.add(Conv2D(128, (3, 3), activation='relu',  kernel_regularizer=l2(2e-4)))
convnet.add(BatchNormalization())
convnet.add(Dropout(0.5))

convnet.add(Conv2D(256, (3, 3), activation='relu',  kernel_regularizer=l2(2e-4)))
# convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.5))

convnet.add(Conv2D(512, (3, 3), activation='relu',  kernel_regularizer=l2(2e-4)))
# convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))

convnet.add(Flatten())
convnet.add(BatchNormalization())
convnet.add(Dense(2048, name="neural_codes", activation="sigmoid", kernel_regularizer=l2(1e-3)))
convnet.summary()

# encode each of the two inputs into a vector with the convnet
encoded_anchor = convnet(anchor_input)
encoded_positive = convnet(positive_input)
encoded_negative = convnet(negative_input)


outputs= concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name="merged_layer")

triplet_net = Model(inputs=[anchor_input, positive_input ,negative_input], outputs=outputs)


triplet_net.compile(loss=triplet_loss, optimizer=keras.optimizers.Adadelta(1.2))

triplet_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 15, 15, 128)       512       
_________________________________________________________________
dropout_16 (Dropout)         (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 13, 13, 128)       147584    
_________________________________________________________________
batch_normalization_21 (Batc (None, 13, 13, 128)       512       
_________________________________________________________________
dropout_17 (Dropout)         (None, 13, 13, 128)       0         
__________

In [0]:
def get_batch(batch_size, X):
    """Create batch of n triplets, half same class, half different class"""
    n_classes, n_examples, w, h, d = X.shape

    categories = np.random.choice(n_classes, size=(batch_size,), replace=False)
    triplets = [np.zeros((batch_size, h, w, d)) for i in range(3)]
    targets = np.zeros((batch_size,))
    for i in range(batch_size):
        category = categories[i]
        idx_1 = np.random.randint(0, n_examples)
        triplets[0][i, :, :, :] = X[category, idx_1].reshape(w, h, d)
        idx_2 = np.random.randint(0, n_examples)
        triplets[1][i, :, :, :] = X[category, idx_2].reshape(w, h, d)
        idx_3 = np.random.randint(0, n_examples)
        category_2 = (category + np.random.randint(1,n_classes)) % n_classes
        triplets[2][i, :, :, :] = X[category_2, idx_3].reshape(w, h, d)
    return triplets, targets

def batch_generator(batch_size, X):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        triplets, targets = get_batch(batch_size, X)
        yield (triplets, targets)

def train_triplet_network(model, X_train, batch_size=64, steps_per_epoch=100, epochs=40):
    model.fit_generator(batch_generator(batch_size, X_train), steps_per_epoch=steps_per_epoch, epochs=epochs)
        

In [33]:
train_triplet_network(triplet_net, x_train_trip, epochs=40)

Epoch 1/40
100/100 [==============================] - 28s 280ms/step - loss: 121.0985
Epoch 2/40
100/100 [==============================] - 19s 193ms/step - loss: 99.3938
Epoch 3/40
100/100 [==============================] - 19s 192ms/step - loss: 99.2640
Epoch 4/40
100/100 [==============================] - 19s 192ms/step - loss: 90.7098
Epoch 5/40
100/100 [==============================] - 19s 192ms/step - loss: 90.7467
Epoch 6/40
100/100 [==============================] - 19s 193ms/step - loss: 87.0404
Epoch 7/40
100/100 [==============================] - 19s 192ms/step - loss: 86.5921
Epoch 8/40
100/100 [==============================] - 19s 192ms/step - loss: 84.3718
Epoch 9/40
100/100 [==============================] - 19s 192ms/step - loss: 85.6403
Epoch 10/40
100/100 [==============================] - 19s 192ms/step - loss: 82.3017
Epoch 11/40
100/100 [==============================] - 19s 192ms/step - loss: 79.1230
Epoch 12/40
100/100 [==============================] - 19s 192

### Task 2.2: One-shot learning with triplet neural codes
**a)**
* Use neural codes from the triplet network with L2-distance to evaluate one-shot learning accuracy for the remaining 20 classes of Cifar-100 with 250 random tasks. I.e. for a given one-shot task, obtain neural codes for the test image as well as the support set. Then pick the image from the support set that is closest (in L2-distance) to the test image as your one-shot prediction.
* Explicitly state the accuracy.

In [0]:
def generate_one_shot_task(N, X):
  #generate positive and negative support sets
  n_classes, n_examples, w, h, d = X.shape


  categories = np.random.choice(range(n_classes), size=(N,), replace=False) #randomly choose N classes(20) from the total number of classes
  indices = np.random.randint(0, n_examples, size=(N,)) #randomly select N indices
  true_category = categories[0] #select the first category as the true categor
  ex_anchor = np.random.choice(n_examples, replace=False, size=(1,)) #randomly choose 1 index for anchor(test) example image

  #select the test image from the training set, from the true category, with index=ex_anchor
  #why do we reshape here: because we will pass inputs to model as 3 arrays, each containing 20 images. The other two arrays will
  #have N distinct images, 1 array from the true category and the other containing 20 images from remaining 19 classes

  test_image = np.asarray([X[true_category, ex_anchor, :, :]]*N).reshape(N, w, h, d)

#   print('shape of test img: ', test_image.shape)

  positive_support_set= X[true_category, indices, :,:]

#   print('shape of positive_support_set', positive_support_set.shape)

  negative_categories= categories[1:]

#   print('shape of negative_categories', negative_categories.shape)

  Nth_category= np.random.choice(negative_categories, replace=False, size=(1,))
#   print('Nth_category', Nth_category)

  negative_categories= np.append(negative_categories, Nth_category)

#   print('shape of negative_categories', negative_categories.shape)

 

  negative_support_set= X[negative_categories, indices, :, :]
#   print('shape of negative_support_set', negative_support_set.shape)
  
  triplets= [test_image, positive_support_set, negative_support_set]
  return triplets
  

In [41]:
#GENERATE 250 ONE-SHOT TASKS


k=250
N=20

#  Extract the dense layer
# nodes_model = Model(inputs=triplet_net.input, outputs=triplet_net.get_layer("merged_layer").output) 

nodes_model = triplet_net
n_correct = 0

for i in range(k):
  inputs = generate_one_shot_task(N, x_test_trip)
  probs = nodes_model.predict(inputs)
#   print('shape probs', probs.shape)
  #  Extract the three embedding vectors
  anchor = probs[:,0:2048]
  pos = probs[:,2048:4096]
  neg = probs[:,4096:6144]
  
  #calculating the accuracy
  
  l2_dist=0
  
  for j in range(N): 
      l2_pos = distance.euclidean(anchor[j], pos[j])
      l2_neg = distance.euclidean(anchor[j], neg[j])
      l2_dist += l2_pos - l2_neg
  if l2_dist > 0:
    n_correct += 1
percent_correct = (100.0*n_correct / k)
print("Got an average of {}% accuracy for {}-way one-shot learning".format(percent_correct, N))


Got an average of 20.0% accuracy for 20-way one-shot learning
